In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd

import importlib
import sys

In [2]:
from dask.distributed import Client

In [3]:
client = Client(n_workers=1, threads_per_worker=8, memory_limit=10e9)
client

/opt/tljh/user/envs/parcels-container_2021.09.29-09ab0ce/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44285 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/workshop007/proxy/44285/status,
Dashboard: /user/workshop007/proxy/44285/status,Workers: 1
Total threads: 8,Total memory: 9.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44191,Workers: 1
Dashboard: /user/workshop007/proxy/44285/status,Total threads: 8
Started: Just now,Total memory: 9.31 GiB
Comm: tcp://134.245.214.97:43613,Total threads: 8
Dashboard: /user/workshop007/proxy/38859/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:38613,


In [4]:
# parameters
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

data_path = "/data/spg_fresh_blob_202104_data/raw/"

interim_data_path = Path('data/interim/endtracks/plusDist/')

sectionPath = Path('data/external/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'

year = 1990

In [5]:
year_str = str(year)

In [6]:
interim_data_filename = "endtracks_20220207_randomvel_mxl_osnap_backwards_"+year_str+"_subset_10percent.nc"

In [7]:
# data_stores_subsets = list(sorted(Path(data_path).glob("*_????_subset.zarr/")))[:use_number_subset_years]
data_stores_subsets = list(sorted(Path(data_path).glob("*_"+year_str+"_subset_10percent.zarr/")))

In [8]:
display(data_stores_subsets)

[PosixPath('/data/spg_fresh_blob_202104_data/raw/tracks_randomvel_mxl_osnap_backward_1990_subset_10percent.zarr')]

In [9]:
ds_subsets = xr.concat(
    [xr.open_zarr(store) for store in data_stores_subsets],
    dim="traj",
)

display(ds_subsets)
print(ds_subsets.nbytes / 1e9, "GiB")

/tmp/ipykernel_2993/237466986.py:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  [xr.open_zarr(store) for store in data_stores_subsets],


<xarray.Dataset>
Dimensions:  (traj: 516415, obs: 731)
Dimensions without coordinates: traj, obs
Data variables:
    lat      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    lon      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    mxl      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    salt     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    temp     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    time     (traj, obs) datetime64[ns] dask.array<chunksize=(16138, 23), meta=np.ndarray>
    uvel     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    vvel     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    z        (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

15.0999746 GiB


## add cumulative track lengths to file

In [10]:
ds_subsets_lat_diff=ds_subsets.lat.diff(dim='obs',n=1)
ds_subsets_lon_diff=ds_subsets.lat.diff(dim='obs',n=1)


In [11]:
# Calculate Great Circle distance between requeted point and 
# matched grid point. 
# Based on https://andrew.hedges.name/experiments/haversine/

# copied from internet

# approximate radius of earth in km
R = 6373000.0

lat1 = xr.ufuncs.radians(ds_subsets.lat)
lon1 = xr.ufuncs.radians(ds_subsets.lon)
lat2 = lat1.shift(obs=1)
lon2 = lon1.shift(obs=1)

dlat = lat2 - lat1
dlon = lon2 - lon1

a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

distance = R * c


In [12]:
ds_subsets = ds_subsets.assign({'dist':distance.cumsum(dim='obs')})

In [13]:
ds_subsets.dist.attrs = {'long_name':'alongtrack distance from OSNAP'
                        ,'units':'metres'}


In [14]:
ds_subsets

<xarray.Dataset>
Dimensions:  (traj: 516415, obs: 731)
Dimensions without coordinates: traj, obs
Data variables:
    lat      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    lon      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    mxl      (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    salt     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    temp     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    time     (traj, obs) datetime64[ns] dask.array<chunksize=(16138, 23), meta=np.ndarray>
    uvel     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    vvel     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    z        (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
    dist     (traj, obs) float32 dask.array<chunksize=(6000, 731), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

## Extract data on osnap line (initialised positions)

In [15]:
ds_subsets_osnap = ds_subsets.isel(obs=0)

## Update some attributes

In [16]:
# ds_subsets_osnap['vol_trans_normal'] = 
ds_subsets_osnap.mxl.attrs = {'units':'m','long_name':'mixed layer depth'}
ds_subsets_osnap.salt.attrs = {'units':'PSU','long_name':'salinity'}
ds_subsets_osnap.temp.attrs = {'units':'degC','long_name':'temperature'}
ds_subsets_osnap.uvel.attrs = {'units':'degrees_east/second','long_name':'u velocity (raw)'}
ds_subsets_osnap.vvel.attrs = {'units':'degrees_north/second','long_name':'v velocity (raw)'}

## Flag tracks by source region and pathway

In [17]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected while they are in a selected area.
    '''
    # particles are selected if they pass through given area.
    position =  ((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                        
    return position > 0.0, position < 0

#### from Labrador sea or from Gulf Stream

In [18]:
# from labrador sea
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-75,-40,40,65)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-100,-58.2,48,52)
ds_in3, ds_notin3 = apply_left_of_line(ds_subsets,-45,-45,60,70)
ds_lab_in = ds_in1*ds_in2*ds_in3
# from west of 60W, south of Flemish Cap (to test path from labrador sea)
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-60,-60,33,63)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-58.2,-100,52,48)
ds_60w_in = ds_in1*ds_in2
# from gulf stream
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-60,-100,33,33)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-44,-44,0,33)
ds_gst_in = ds_in1 * ds_in2


In [19]:
# check trajectory routes
LabCu = ds_lab_in.max("obs")
LC60W = ds_60w_in.max("obs")
GulfS = ds_gst_in.max("obs")

# check when lef lab sea,crossed 60w or gulf stream. defaults to zero
LabCu_exit_index = ds_lab_in.argmax(axis=1)
LC60W_exit_index = ds_60w_in.argmax(axis=1)
GulfS_exit_index = ds_gst_in.argmax(axis=1)

LabCu_exit_index = LabCu_exit_index.where(LabCu_exit_index > 0,len(ds_subsets.obs)-1)
LC60W_exit_index = LC60W_exit_index.where(LC60W_exit_index > 0,len(ds_subsets.obs)-1)
GulfS_exit_index = GulfS_exit_index.where(GulfS_exit_index > 0,len(ds_subsets.obs)-1)

In [20]:
# check if most recently left Lab Sea
LabCu_is_source = (LabCu_exit_index < GulfS_exit_index)
GulfS_is_source = (LabCu_exit_index > GulfS_exit_index)
LC60W_is_path = (LabCu_exit_index > LC60W_exit_index).where(LabCu_is_source,False)

In [21]:
LCdir_is_path = LabCu_is_source.where(LC60W_is_path==False,False)
other_is_source = (LabCu_is_source==False).where(GulfS_is_source == False,False)

flag particles on osnap line by origin

In [22]:
ds_subsets_osnap = ds_subsets_osnap.assign({'LabCu_is_source':LabCu_is_source})
ds_subsets_osnap = ds_subsets_osnap.assign({'LC60W_is_path':LC60W_is_path})
ds_subsets_osnap = ds_subsets_osnap.assign({'LCdir_is_path':LCdir_is_path})
ds_subsets_osnap = ds_subsets_osnap.assign({'GulfS_is_source':GulfS_is_source})
ds_subsets_osnap = ds_subsets_osnap.assign({'other_is_source':other_is_source})


flag by pathway for Labrador Current parcels

In [23]:
ds_subsets_osnap.LabCu_is_source.attrs = {'long_name':'flag from Labrador Current'}
ds_subsets_osnap.LC60W_is_path.attrs   = {'long_name':'flag from LC via 60W'}
ds_subsets_osnap.LCdir_is_path.attrs   = {'long_name':'flag from LC direct'}
ds_subsets_osnap.GulfS_is_source.attrs = {'long_name':'flag from Gulf Stream'}
ds_subsets_osnap.other_is_source.attrs = {'long_name':'flag source not found'}

## Find the 'obs' index of the point where parcel leaves the source region

### known source regions

In [24]:
# test individaul positions to see when the source was left
# defaults to zero if particular source was not on track
LabCu_exit_index = (ds_lab_in.where(ds_subsets_osnap.LabCu_is_source,False)).argmax(axis=1)
GulfS_exit_index = (ds_gst_in.where(ds_subsets_osnap.GulfS_is_source,False)).argmax(axis=1)

# combine in to one array. Nonzero numbers should not overlap for 
# Lab current and Gulf Stream
exit_index = LabCu_exit_index + GulfS_exit_index
# convert zeros to max dim obs
exit_index = exit_index.where(exit_index > 0,len(ds_subsets.obs)-1)

### index last non nan value for 'other' parcels

In [25]:
a = ds_subsets.lat  # just a random selection of variable, nans the same for all variables
b = (~np.isnan(a)).cumsum(dim='obs').argmax(dim='obs') # finds last non-nan in dim 'obs'. nicked from stackoverflow search
exit_index = xr.ufuncs.minimum(exit_index,b).compute()

### extract source positions from full array

In [26]:
ds_subsets_sourc = ds_subsets.isel(traj=xr.DataArray(range(len(ds_subsets.traj)),dims='traj'),obs=exit_index)

distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)


add the source and pathway flags to ds_subsets_sourc to match for xr.concat

In [27]:
# flag particles on osnap line by origin and pathway
ds_subsets_sourc = ds_subsets_sourc.assign({'LabCu_is_source':LabCu_is_source})
ds_subsets_sourc = ds_subsets_sourc.assign({'LC60W_is_path':LC60W_is_path})
ds_subsets_sourc = ds_subsets_sourc.assign({'LCdir_is_path':LCdir_is_path})
ds_subsets_sourc = ds_subsets_sourc.assign({'GulfS_is_source':GulfS_is_source})
ds_subsets_sourc = ds_subsets_sourc.assign({'other_is_source':other_is_source})


In [28]:
ds_subsets_sourc.LabCu_is_source.attrs = {'long_name':'flag from Labrador Current'}
ds_subsets_sourc.LC60W_is_path.attrs   = {'long_name':'flag from LC via 60W'}
ds_subsets_sourc.LCdir_is_path.attrs   = {'long_name':'flag from LC direct'}
ds_subsets_sourc.GulfS_is_source.attrs = {'long_name':'flag from Gulf Stream'}
ds_subsets_sourc.other_is_source.attrs = {'long_name':'flag source not found'}

### combine source and osnap positions and characteristics

In [29]:
ds_subsets_paths = xr.concat([ds_subsets_osnap,
                             ds_subsets_sourc],
                             dim='ends')

### flag particles entering from north from Greenland Sea or Davis Strait

In [30]:
b = b.compute()

In [31]:
ds_subsets_domexi = ds_subsets.isel(traj=xr.DataArray(range(len(ds_subsets.traj)),dims='traj'),obs=b)

distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [32]:
# counts how many times enters or leaves lab sea
spgnoloop = (abs(ds_lab_in.astype(int).diff(dim='obs')).sum(dim='obs')<3).compute()

distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)


In [33]:
# from Hudson Bay
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-68,-68,33,63)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-95,-60,52,52)
ds_hud_in = ds_in1*ds_in2

HudBa = ds_hud_in.max("obs")


In [34]:
HudBa.data.compute()

array([False, False, False, ..., False, False, False])

In [35]:
Green_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source &
                  (ds_subsets_domexi.lat > 65) & 
                  (ds_subsets_domexi.lon > -44) &
                  spgnoloop)
Davis_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source &
                  (ds_subsets_domexi.lat > 65) & 
                  (ds_subsets_domexi.lon < -44) &
                  spgnoloop)
Hudba_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source & 
                  HudBa &
                  spgnoloop)

In [36]:
Green_is_source = xr.concat([(Green_is_source),(Green_is_source)],dim='ends')
ds_subsets_paths = ds_subsets_paths.assign({'Green_is_source':Green_is_source})
Davis_is_source = xr.concat([(Davis_is_source),(Davis_is_source)],dim='ends')
ds_subsets_paths = ds_subsets_paths.assign({'Davis_is_source':Davis_is_source})
Hudba_is_source = xr.concat([(Hudba_is_source),(Hudba_is_source)],dim='ends')
ds_subsets_paths = ds_subsets_paths.assign({'Hudba_is_source':Hudba_is_source})


add the source and pathway flags to ds_subsets_sourc to match for xr.concat

In [37]:
ds_subsets_paths.Davis_is_source.attrs = {'long_name':'flag from Davis Strait'}
ds_subsets_paths.Green_is_source.attrs = {'long_name':'flag from Greenland Sea'}
ds_subsets_paths.Hudba_is_source.attrs = {'long_name':'flag from Hudson Bay'}


## We want to test for tracks which route north of osnap line between source and final times

This is because a common strategy is to remove these from analysis and only consider the 'direct' paths.

### Test particle positions

### section position data

In [38]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))

#### south/north of osnap-e

In [39]:
# do north and south separately because of missing values

south = xr.Dataset()
north = xr.Dataset()
epsilon = 0.05
for i in range(len(lonlat.lon)-1):
    south['subsect'+str(i)],north['subsect'+str(i)] = apply_left_of_line(ds_subsets,lonlat.lon[i+1],lonlat.lon[i],lonlat.lat[i+1]+epsilon,lonlat.lat[i]+epsilon)

# check in osnap east 
south_oe,north_oe = apply_left_of_line(ds_subsets,-44,-44,30,60)

# south_a = south.subsect0 + south.subsect1 + south.subsect2 
# south_b = south.subsect3 * south.subsect4 * south.subsect5
south_b = south.subsect4 * south.subsect5
south_c = south.subsect6 + south.subsect7 + south.subsect8 
south_d = south.subsect8 * south.subsect9 * south.subsect10 * south.subsect11 
# south_e = south.subsect12
# south_all = south_a * south_c * south_e * (south_b + south_d)
south_all = south_oe + (south_c * (south_b + south_d))

# north_a = north.subsect0 * north.subsect1 * north.subsect2 
# north_b = north.subsect3 + north.subsect4 + north.subsect5
north_b = north.subsect4 + north.subsect5
north_c = north.subsect6 * north.subsect7 * north.subsect8 
north_d = north.subsect8 + north.subsect9 + north.subsect10 + north.subsect11
# north_e = north.subsect12
# north_all = north_a + north_c + north_e + (north_b * north_d)
north_all = north_oe * (north_c  + (north_b * north_d))


In [40]:
north_all = north_all.reset_coords(drop=True)
south_all = south_all.reset_coords(drop=True)

In [41]:
# test individual positions to see when the parcel was first (in 'obs', last in time) north of osnap line
# defaults to zero if particular source was not on track
north_osnap_index = (north_all).argmax(axis=1)
# convert zeros to max dim obs
north_osnap_index = north_osnap_index.where(north_osnap_index > 0,len(ds_subsets.obs)-1)

Check if found north of osnap e between leaving source and arriving at osnap and flag to ds_subsets_paths

In [42]:
north_osnap = xr.concat([(north_osnap_index < exit_index),(north_osnap_index < exit_index)],dim='ends')
ds_subsets_paths = ds_subsets_paths.assign({'north_of_osnap':north_osnap})
ds_subsets_paths.north_of_osnap.attrs = {'long_name':'flag path goes north of osnap-e'}
display(ds_subsets_paths)

<xarray.Dataset>
Dimensions:          (ends: 2, traj: 516415)
Dimensions without coordinates: ends, traj
Data variables: (12/19)
    lat              (ends, traj) float32 dask.array<chunksize=(1, 6000), meta=np.ndarray>
    lon              (ends, traj) float32 dask.array<chunksize=(1, 6000), meta=np.ndarray>
    mxl              (ends, traj) float32 dask.array<chunksize=(1, 6000), meta=np.ndarray>
    salt             (ends, traj) float32 dask.array<chunksize=(1, 6000), meta=np.ndarray>
    temp             (ends, traj) float32 dask.array<chunksize=(1, 6000), meta=np.ndarray>
    time             (ends, traj) datetime64[ns] dask.array<chunksize=(1, 16138), meta=np.ndarray>
    ...               ...
    GulfS_is_source  (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
    other_is_source  (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
    Green_is_source  (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
    Davis_is_source  (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
    Hudba_is_source  (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
    north_of_osnap   (ends, traj) bool dask.array<chunksize=(1, 6000), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

## Output to netcdf

In [43]:
ds_subsets_paths.to_netcdf(project_path / interim_data_path / interim_data_filename)

distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 36% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
/opt/tljh/user/envs/parcels-container_2021.09.29-09ab0ce/lib/python3.9/site-packages/distributed/worker.py:3877: UserWarning: Large object of size 2.34 MiB detected in task graph: 
  ([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1 ... 8f16c8', 86)"])
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(
distributed.utils_perf - WARNING

In [2]:
conda list

# packages in environment at /opt/tljh/user/envs/parcels-container_2021.09.29-09ab0ce:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
ansiwrap                  0.8.4                      py_0    conda-forge
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
asciitree                 0.3.3                      py_2    conda-forge
async_generator           1.10                       py_0    conda-forge
attrs                     21.2.0             pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports                 1.0                        py_2    conda-forge
backports.functools_lru_cache 1.6.4              pyhd8ed1ab_0    conda-forge
basemap                   1.2.2            py39h523bb60_3    conda-forge
binutils_impl_linux-64    2.36.1     